In [12]:
from seaborn import load_dataset

In [13]:
tips = load_dataset('tips')

In [14]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [15]:
tips['tip_pct'] = tips['tip']/tips['total_bill']

컬럼에 따라 다른 함수를 사용해서 집계를 수행하거나 여러개의 함수를 한번에 적용하고 싶다면 이를 쉽고 간단하게 수행할 수 있다.

In [22]:
tips.tail()

,total_bill,tip,sex,smoker,day,time,size,tip_pct
239,29.03,5.92,Male,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Female,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Male,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,Male,No,Sat,Dinner,2,0.098204
243,18.78,3.00,Female,No,Thur,Dinner,2,0.159744


In [49]:
grouped = tips.groupby(['sex','smoker'])

In [50]:
grouped.count()

total_bill  tip  day  time  size  tip_pct
sex    smoker                                           
Male   Yes             60   60   60    60    60       60
       No              97   97   97    97    97       97
Female Yes             33   33   33    33    33       33
       No              54   54   54    54    54       54

In [17]:
grouped_pct = grouped['tip_pct']

In [23]:
pd.DataFrame(grouped_pct.agg('mean'))

tip_pct
sex    smoker          
Male   Yes     0.152771
       No      0.160669
Female Yes     0.182150
       No      0.156921

만약 함수의 목록이나 함수 이름들을 넘기면 함수 이름을 컬럼으로 하는 DataFrame을 얻게 된다.

In [20]:
grouped_pct.agg(['mean','std',peak_to_peak])

mean       std  peak_to_peak
sex    smoker                                  
Male   Yes     0.152771  0.090588      0.674707
       No      0.160669  0.041849      0.220186
Female Yes     0.182150  0.071595      0.360233
       No      0.156921  0.036421      0.195876

GroupBy 객체에서 자동으로 지정되는 컬럼 이름을 그대로 쓰지 않아도 된다.<br/>
lambda 함수는 이름(함수 이름은 __name__ 속성으로 확인이 가능하다)이 `lambda` 인데, 이를 그대로 쓰면 오히려 알아보기 힘들다.<br/>
이름과 함수가 담긴 튜플의 리스트를 넘기면 각 튜플에서 첫 번째 원소는 DataFrame에서 컬럼의 이름으로 사용된다<br/>
(2개의 튜플을 가지는 리스트가 순서대로 연결된다.)

In [25]:
grouped_pct.agg([('foo','mean'),('boo','std')])

foo       boo
sex    smoker                    
Male   Yes     0.152771  0.090588
       No      0.160669  0.041849
Female Yes     0.182150  0.071595
       No      0.156921  0.036421

In [26]:
grouped_pct.agg([('foo','mean'),('boo',np.std)])

foo       boo
sex    smoker                    
Male   Yes     0.152771  0.090588
       No      0.160669  0.041849
Female Yes     0.182150  0.071595
       No      0.156921  0.036421

DataFrame은 컬럼마다 다른 함수를 적용하거나 여러 개의 함수를 모든 컬럼에 적용할 수 있다.<br/>
'tip_pct'와 'total_bill'컬럼에 세 가지 통계를 계산한다고 가정하자.

In [27]:
functions = ['count','mean','max']

In [28]:
result = grouped['tip_pct','total_bill'].agg(functions)

In [29]:
result

tip_pct                     total_bill                  
                count      mean       max      count       mean    max
sex    smoker                                                         
Male   Yes         60  0.152771  0.710345         60  22.284500  50.81
       No          97  0.160669  0.291990         97  19.791237  48.33
Female Yes         33  0.182150  0.416667         33  17.977879  44.30
       No          54  0.156921  0.252672         54  18.105185  35.83

위애서 확인할 수 있듯이 반환된 DF은 계층적인 컬럼을 가지고 있으며, 이는 각 컬럼을 따로 계산한 다음 `concat` 메서드를 이용해서 `keys`인자로 컬럼 이름을 넘겨서 이어붙인 것과 동일하다.

In [30]:
result['tip_pct']

count      mean       max
sex    smoker                           
Male   Yes        60  0.152771  0.710345
       No         97  0.160669  0.291990
Female Yes        33  0.182150  0.416667
       No         54  0.156921  0.252672

이처럼 컬럼 이름과 메서드가 담긴 튜플의 리스트를 넘기는 것도 가능하다.

In [42]:
ftuples = ['mean','var']

In [43]:
ftuples = [('AAAAAAAAA','mean'),('BBBBBBBB','var')]

In [44]:
grouped['tip_pct','total_bill'].agg(ftuples)

tip_pct           total_bill           
              AAAAAAAAA  BBBBBBBB  AAAAAAAAA   BBBBBBBB
sex    smoker                                          
Male   Yes     0.152771  0.008206  22.284500  98.244673
       No      0.160669  0.001751  19.791237  76.152961
Female Yes     0.182150  0.005126  17.977879  84.451517
       No      0.156921  0.001327  18.105185  53.092422

In [48]:
grouped.mean()

total_bill       tip      size   tip_pct
sex    smoker                                          
Male   Yes      22.284500  3.051167  2.500000  0.152771
       No       19.791237  3.113402  2.711340  0.160669
Female Yes      17.977879  2.931515  2.242424  0.182150
       No       18.105185  2.773519  2.592593  0.156921

컬럼마다 다른 함수를 적용하고 싶다면 agg 메서드에 컬럼 이름에 대응하는 함수가 들어있는 사전을 넘기면 된다.

In [53]:
grouped.agg({'tip':'max',
            'size':'sum'})

tip  size
sex    smoker            
Male   Yes     10.0   150
       No       9.0   263
Female Yes      6.5    74
       No       5.2   140

만약 단 하나의 컬럼이라도 여러개의 함수가 적용된다면 DF은 계층적인 컬럼을 가진다.